In [1]:
import ee
import geemap
print(f"Earth Engine API version: {ee.__version__}")
print(f"Geemap version: {geemap.__version__}")
import pprint
from pprint import pprint
import matplotlib.pyplot as plt #para os gráficos de barras
import geemap.colormaps as cm #para cm.get_palette
import csv #para salvar dados no formato
import pandas as pd
import numpy as np
import ee_jupyter
import matplotlib.colors as mcolors

Earth Engine API version: 1.7.0
Geemap version: 0.36.6


Para se autenticar junto à conta no Google Earth Engine

In [2]:
# Disparar o pedido de autenticação
ee.Authenticate()
# Iniciar a bibllioteca do EE
ee.Initialize()


In [3]:
# Recuperando ativos 

# Shapefiles de Londrina
per_urb = ee.FeatureCollection('users/fjcosta/LIM_Perimetro_Urbano_V3_wsg84')
per_mun = ee.FeatureCollection('users/fjcosta/perimetro_municipio_wsg84')

# Valores preditos
asset_list = ee.data.listAssets("users/fjcosta/pred_estimados")
asset_names = [d['name'] for d in asset_list['assets']]

# Create an ImageCollection from asset names
preditos = ee.ImageCollection(asset_names)

# Imprimir o nome das bandas da primeira imagem
print(preditos.first().bandNames().getInfo()) #banda com os valores:b1

# Função para alterar o nome da banda dos valores e incluir campos com datas
def transform_preditos(image):
    # Mudar o nome
    image = image.select(['b1'], ['preditos'])

    # Usar como tempo a data desse campo
    date_acquired = ee.Date(image.get('DATE_ACQUIRED'))
    image = image.set('system:time_start', date_acquired.millis())
    image = image.set('system:time_end', date_acquired.millis())

    # Adicionar a banda com o valor predito transformado à escala original
    exp_band = image.select('preditos').exp()
    image = image.addBands(exp_band.rename('expPreditos'))

    return image

# Aplicar a função à coleção
preditos = preditos.map(transform_preditos)

# Função para clipar
def clp(img):
  return img.clip(per_urb)


# Clipar
preditos = preditos.map(clp)

# Print the 'preditos' collection and the nominal scale of the first image
display('Coleção de valores preditos', preditos.getInfo())

# Escala nominal da primeira imagem
preditos_res = preditos.first().projection().nominalScale()
display( 'Escala espacial dos valores preditos ',preditos_res.getInfo()) #52.33


['b1']


'Coleção de valores preditos'

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'preditos',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'dimensions': [355, 466],
     'crs': 'EPSG:4326',
     'crs_transform': [0.0004890000000000168,
      0,
      -51.250404490106504,
      0,
      -0.0004520000000000008,
      -23.231734368843902]},
    {'id': 'expPreditos',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'dimensions': [355, 466],
     'crs': 'EPSG:4326',
     'crs_transform': [0.0004890000000000168,
      0,
      -51.250404490106504,
      0,
      -0.0004520000000000008,
      -23.231734368843902]}],
   'version': 1697375392760389,
   'id': 'users/fjcosta/pred_estimados/pred_loci2000_raster_log_wgs84',
   'properties': {'system:time_start': 962409600000,
    'system:time_end': 962409600000,
    'system:index': '0',
    'DATE_ACQUIRED': '2000-07-01',
    'system:footprint': {'type': 'LinearRing',
     'coordinates': [[-51.2

'Escala espacial dos valores preditos '

52.33531687634693

In [4]:
# Selecionar uma imagem específica
pred_07_2021 = preditos.select('expPreditos').filter(ee.Filter.eq('DATE_ACQUIRED', '2021-07-01')).first()

# Extrair a data
data_acquired = pred_07_2021.get('DATE_ACQUIRED').getInfo()

# Calcular min e max da banda expPreditos
stats = pred_07_2021.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=per_urb,
    scale=preditos_res.getInfo(),
    maxPixels=1e9
).getInfo()

min_value = stats['expPreditos_min']
max_value = stats['expPreditos_max']

# Exibir os valores
print(f"Data: {data_acquired}")
print(f"Mínimo: {min_value:.2f}")
print(f"Máximo: {max_value:.2f}")


Data: 2021-07-01
Mínimo: 36.60
Máximo: 3776.16


In [5]:
# Definir paleta
# predPalette = cm.get_palette('plasma', n_class=256)
predPalette = cm.get_palette('OrRd', n_class=256)


# Criar visualização dinâmica
predVis = {
    'min': min_value,
    'max': max_value,
    'palette': predPalette,
    'bands': 'expPreditos'
}

# Criar mapa
mapa = geemap.Map()
mapa.setCenter(-51.1662, -23.3197, 12)
mapa.addLayer(pred_07_2021.clip(per_urb), predVis, 'Preditos')
mapa.add_colorbar_branca(
    colors=predPalette,
    vis_params=predVis,
    vmin=round(min_value, 2),
    vmax=round(max_value, 2),
    layer_name=data_acquired,
    caption=f'Median urban vacant parcels unit values predicted for: {data_acquired} (R$/m²)'    
)

# Salvar o mapa como arquivo HTML
html_file_path = '/Users/fjcosta/Documents/landCoverlandValue/landvalue/temporal/jul_2021GEE.html'
mapa.to_html(html_file_path)
mapa

Map(center=[-23.3197, -51.1662], controls=(WidgetControl(options=['position', 'transparent_bg'], position='top…

In [6]:
import geemap.foliumap as geemap
import geemap.colormaps as cm
import ee
import folium

# Definir paleta
predPalette = cm.get_palette('OrRd', n_class=256)

# Obter todas as datas disponíveis
dates_list = preditos.aggregate_array('DATE_ACQUIRED').getInfo()
print(f"Total de datas disponíveis: {len(dates_list)}")
print(f"Datas: {dates_list}")

# Diretório de saída
output_dir = '/Users/fjcosta/Documents/landCoverlandValue/landvalue/temporal/gee/'

# Processar cada data individualmente
for date_str in dates_list:
    print(f"\nProcessando: {date_str}")
    
    # Filtrar imagem para esta data
    img = preditos.select('expPreditos').filter(
        ee.Filter.eq('DATE_ACQUIRED', date_str)
    ).first()
    
    # Calcular min e max específicos desta data
    stats = img.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=per_urb,
        scale=preditos_res.getInfo(),
        maxPixels=1e9
    ).getInfo()
    
    min_value = stats['expPreditos_min']
    max_value = stats['expPreditos_max']
    
    print(f"  Min: {min_value:.2f}, Max: {max_value:.2f}")
    
    # Visualização específica para esta data
    predVis = {
        'min': min_value,
        'max': max_value,
        'palette': predPalette,
        'bands': 'expPreditos'
    }
    
    # Criar mapa
    mapa = geemap.Map()
    mapa.setCenter(-51.1662, -23.3197, 12)   
    
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri WorldImagery',
        overlay=False,
        control=True,
        show=True
    ).add_to(mapa)
    
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri WorldStreetMap',
        overlay=False,
        control=True,
        show=False
    ).add_to(mapa)
    
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri WorldTopoMap',
        overlay=False,
        control=True,
        show=False
    ).add_to(mapa)
    
    # Adicionar camada Earth Engine
    mapa.addLayer(img.clip(per_urb), predVis, date_str)
    
    # Adicionar LayerControl
    folium.LayerControl(collapsed=False).add_to(mapa)
    
    # Criar gradiente para legenda
    n_colors = 256
    indices = [int(i * (len(predPalette) - 1) / (n_colors - 1)) for i in range(n_colors)]
    selected_colors = [predPalette[i] for i in indices]
    gradient = ', '.join([f'#{c}' for c in selected_colors])
    
    # Salvar primeiro
    safe_date = date_str.replace('-', '_').replace('/', '_')
    html_file_path = f'{output_dir}pred_{safe_date}.html'
    mapa.save(html_file_path)
    
    # Agora adicionar o HTML customizado diretamente no arquivo
    with open(html_file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # HTML da legenda + controle de opacidade (JavaScript corrigido)
    legend_and_opacity_html = f'''
    <div id="legend-box" style="position: fixed; 
                bottom: 50px; left: 50px; 
                width: 350px;
                background-color: white; 
                z-index: 9999; 
                border: 2px solid #333; 
                border-radius: 5px; 
                padding: 15px;
                box-shadow: 0 0 15px rgba(0,0,0,0.2);">
        <p style="margin: 0 0 10px 0; font-weight: bold; font-size: 13px; line-height: 1.5; color: #333;">
            Median urban vacant parcels unit values<br>
            predicted for: {date_str} (R$/m²)
        </p>
        <div style="background: linear-gradient(to right, {gradient}); 
                    height: 25px; 
                    width: 100%; 
                    border: 1px solid #666;
                    border-radius: 3px;"></div>
        <div style="display: flex; 
                    justify-content: space-between; 
                    margin-top: 8px; 
                    font-size: 11px;
                    color: #333;">
            <span>{min_value:.2f}</span>
            <span>{max_value:.2f}</span>
        </div>
        
        <div style="margin-top: 15px; padding-top: 15px; border-top: 1px solid #ccc;">
            <label for="opacity-slider" style="font-weight: bold; font-size: 12px; color: #333; display: block; margin-bottom: 5px;">
                Layer Opacity: <span id="opacity-value">100</span>%
            </label>
            <input type="range" id="opacity-slider" min="0" max="100" value="100" 
                   style="width: 100%; height: 20px; cursor: pointer;">
        </div>
    </div>
    
    <script>
    window.addEventListener('load', function() {{
        console.log('Opacity control loaded');
        
        const slider = document.getElementById('opacity-slider');
        const valueDisplay = document.getElementById('opacity-value');
        
        if (slider && valueDisplay) {{
            slider.addEventListener('input', function() {{
                const opacity = this.value / 100;
                valueDisplay.textContent = this.value;
                
                console.log('Setting opacity to:', opacity);
                
                // Método 1: Tentar pegar pela classe leaflet-tile-pane
                const tileLayers = document.querySelectorAll('.leaflet-tile-pane');
                tileLayers.forEach(function(pane) {{
                    // Pegar apenas as camadas filhas
                    const children = pane.children;
                    for (let i = 0; i < children.length; i++) {{
                        // Verificar se é uma camada overlay (não basemap)
                        if (children[i].style.zIndex > 200) {{
                            children[i].style.opacity = opacity;
                            console.log('Applied opacity to tile pane child');
                        }}
                    }}
                }});
                
                // Método 2: Pegar pelo overlay-pane
                const overlayPanes = document.querySelectorAll('.leaflet-overlay-pane');
                overlayPanes.forEach(function(pane) {{
                    pane.style.opacity = opacity;
                    console.log('Applied opacity to overlay pane');
                }});
                
                // Método 3: Pegar diretamente as imagens do Earth Engine
                const allImages = document.querySelectorAll('img');
                allImages.forEach(function(img) {{
                    if (img.src.includes('earthengine.googleapis.com')) {{
                        img.style.opacity = opacity;
                        console.log('Applied opacity to EE image');
                    }}
                }});
            }});
        }} else {{
            console.error('Slider or value display not found');
        }}
    }});
    </script>
    '''
    
    # Inserir antes de </body>
    html_content = html_content.replace('</body>', legend_and_opacity_html + '</body>')
    
    # Salvar novamente
    with open(html_file_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"  ✓ Salvo: {html_file_path}")

print(f"\n✓ Total de mapas gerados: {len(dates_list)}")
print(f"✓ Diretório: {output_dir}")

Total de datas disponíveis: 22
Datas: ['2000-07-01', '2001-07-01', '2002-07-01', '2003-07-01', '2004-07-01', '2005-07-01', '2006-07-01', '2007-07-01', '2008-07-01', '2009-07-01', '2010-07-01', '2011-07-01', '2012-07-01', '2013-07-01', '2014-07-01', '2015-07-01', '2016-07-01', '2017-07-01', '2018-07-01', '2019-07-01', '2020-07-01', '2021-07-01']

Processando: 2000-07-01
  Min: 7.47, Max: 167.59
  ✓ Salvo: /Users/fjcosta/Documents/landCoverlandValue/landvalue/temporal/gee/pred_2000_07_01.html

Processando: 2001-07-01
  Min: 8.78, Max: 189.97
  ✓ Salvo: /Users/fjcosta/Documents/landCoverlandValue/landvalue/temporal/gee/pred_2001_07_01.html

Processando: 2002-07-01
  Min: 10.21, Max: 214.35
  ✓ Salvo: /Users/fjcosta/Documents/landCoverlandValue/landvalue/temporal/gee/pred_2002_07_01.html

Processando: 2003-07-01
  Min: 11.73, Max: 240.60
  ✓ Salvo: /Users/fjcosta/Documents/landCoverlandValue/landvalue/temporal/gee/pred_2003_07_01.html

Processando: 2004-07-01
  Min: 13.40, Max: 268.66
  ✓ 

Manipulação para se plotar alguns shapefiles úteis sem prenchimento interno
(os polígonos alta_densidade, baixa_densidade, vale- agua e terra não foram carregados)

In [18]:
# Definindo o perímetro sem preenchimento interno
# Perímetro municipal
outline_per_urb = per_urb.style(
    **{
        'fillColor': '00000000',  # Transparent fill color (no fill)
        'color': 'FF0000',       # Outline color (red)
        'width': 2                # Outline width
    }
)

# Perímetro do município
outline_per_mun = per_mun.style(
    **{
        'fillColor': '00000000',  # Transparent fill color (no fill)
        'color': 'FF0000',       # Outline color (red)
        'width': 2                # Outline width
    }
)




Coleção de imagens do satélite Landast 7 (1999 - )
This dataset contains atmospherically corrected surface reflectance and land surface temperature derived from the data produced by the Landsat 7 ETM+ sensor.

In [19]:
# Importando a coleção USGS Landsat 7 Collection 2 Surface reflectance para uso em estudo da cobertura do solo
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")


Os produtos Landsat Nível 2 são escritos como números inteiros escalonados para nos permitir converter os dados de ponto flutuante em número inteiro para entrega. Na maioria dos casos, eles são gravados em um número inteiro de 16 bits, o que economiza espaço em disco e proporciona tempos de download mais rápidos.   

Cada pixel de ponto flutuante tem um deslocamento aplicado e então multiplicado por um ganho para trazer o valor para o intervalo inteiro de 16 bits (ou inteiro sem sinal). Esses valores são chamados de números inteiros escalonados. Para permitir que o usuário retorne os dados ao seu valor original de ponto flutuante, um fator de escala e um deslocamento são fornecidos para cada banda. O fator de escala deve ser aplicado a cada pixel e então o deslocamento adicionado (ou seja, Número Digital (DN) * fator de escala + deslocamento).  

Você pode aplicar um fator de escala aos produtos científicos por meio de um script de dados, de cálculo manual ou em determinados programas de software.
[cf. em USGS](https://www.usgs.gov/faqs/how-do-i-use-a-scale-factor-landsat-level-2-science-products)

In [20]:
# Fatores de escala (GEE website)

def applyScaleFactors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_band = image.select('ST_B.').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_band, None, True)

# Aplicar a função à coleção l7

l7 = l7.map(applyScaleFactors)



In [21]:
# Função para renomear as bandas de interesse do Landsat 7 ETM C02 para ficar igual (nas bandas iguais) aos nomes da coleção
# Landsat 7 Collection 2 Tier 1 Raw Scenes

def renameEtm(img):
    return img.select(
        ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'ST_B6' , 'QA_PIXEL'],
        ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6', 'pixel_qa'])


A identificação de nuvens, sombras de nuvens e neve em imagens ópticas é muitas vezes um passo necessário para a sua utilização. Recentemente, um novo programa (chamado Fmask) projetado para realizar essas tarefas foi introduzido para uso com imagens de Landsats 4–7 (Zhu & Woodcock, 2012). Neste artigo, existem os seguintes: (1) melhorias no algoritmo Fmask para Landsats 4–7; (2) uma nova versão para uso com Landsat 8 que aproveita a nova banda cirrus; e (3) um algoritmo protótipo para imagens do Sentinel 2 .[cf. em Zhu et al, 2015:](https://www.sciencedirect.com/science/article/abs/pii/S0034425714005069?via%3Dihub)

In [22]:
# Função para mascarar pixels associados a sombras ou nuvens que podem interferir na análise (remove o pixel e fica em branco a área dele)
# Zhu et al., 2015

def fmask(img):
    # Definir os bits para nuvens e sombras de nuvens
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5

    # Selecionar a banda pixel_qa
    qa = img.select('pixel_qa')

    # Criar a máscara usando operações bitwise
    # e deixar os pixels muito brilhantes 'transparentes'
    mask = (
        qa.bitwiseAnd(cloudShadowBitMask).eq(0)
        .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
        .And(img.select('B1').lt(0.2))  # Excluir pixels saturados (ajuste conforme necessário)
    )


    # Aplicar a máscara à imagem
    return img.updateMask(mask)

Landsat Collection 2  marca o segundo grande evento de reprocessamento do arquivo USGS Landsat Level-1, resultando em diversas melhorias de produto que aproveitam os avanços recentes no processamento de dados, desenvolvimento de algoritmos e recursos de acesso e distribuição de dados. Uma característica principal da Coleção 2 é a melhoria substancial na precisão absoluta da geolocalização do conjunto de dados de referência terrestre global usado no fluxo de processamento do Landsat Nível 1. Além disso, a Coleção 2 inclui fontes globais atualizadas de modelagem de elevação digital, atualizações de calibração e validação, bem como refletância de superfície global de nível 2 e produtos baseados em cenas de temperatura de superfície de 1982 até o presente. A coleção 2 contém todos os sensores Landsat Landsat 1-9.[cf. em USGS:](https://www.usgs.gov/landsat-missions/landsat-collections)

In [23]:
# Filtro que retorna os TILES que, de algum modo, estão sobre a 'aoi'
# seleciona o intervalo de tempo desejado (menos nuvens)
# cobertura máxima de nuvens admitida e qualidade da imagem

colFilter = ee.Filter.And(
    ee.Filter.bounds(per_urb),  # esse filtro retorna apenas os TILES que, de algum modo, possuem alguma parte sobre 'per_urb'
    ee.Filter.calendarRange(1, 365, 'day_of_year'),  # esse filtro permite selecionar diferentes períodos do ano, se necessário, para facilitar remoção de nuvens
    ee.Filter.date('2000-01-01', '2000-12-31'), # filtro temporal anula
    ee.Filter.lt('CLOUD_COVER', 10), # filtro de cobertura de nuvens menor que 10%
    ee.Filter.lt('GEOMETRIC_RMSE_MODEL', 10), # fixa um limite para o erro de uma das correções pelas quais as imagens de satélite passam
    ee.Filter.Or(
        ee.Filter.eq('IMAGE_QUALITY', 9),
        ee.Filter.eq('IMAGE_QUALITY_OLI', 9))
)

In [24]:
# Função que prepara as imagens da coleção do Landsat 7 ETM e renomeia as bandas, mascara os pixels muito nublados

def prepEtm(img):
    orig = img
    img = renameEtm(img)
    img = fmask(img)
    return ee.Image(img.copyProperties(orig, orig.propertyNames()))

In [25]:
# Aplicando o filtro colFilter à coleção total e depois alicar a função de renomear as bandas e mascaram os pixels

l72000= l7.filter(colFilter).map(prepEtm)

# Tamanho da coleção (número de imagens)
num_images = l72000.size()

# Imprimir o resultado
print("Número de imagens na coleção após filtragem:", num_images.getInfo()) # 6 imagens entre 2000-01-01 e 2000-12-31, com baixa cobertura de nuvens e com pixels mascarados

# Selecionar a primeira imagem na coleção l7 (você pode escolher outra se desejar)
primeira_imagem = l72000.first()

# Obter os nomes das bandas da primeira imagem
nomes_das_bandas = primeira_imagem.bandNames()

# Imprimir os nomes das bandas
display("Nomes das bandas na primeira imagem:", nomes_das_bandas.getInfo())


Número de imagens na coleção após filtragem: 6


'Nomes das bandas na primeira imagem:'

['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6', 'pixel_qa']

Função disponibilizada pelo Google Earth Engine para criar um compósito com controle sobre a imagem composta final em termos de cobertura de nuvens e também os valores espectrais medianos de todas as imagens do período selecionado

In [26]:
# Criar uma imagem composta (composite) mediana

l72000Composite = l72000.median()


# 1 imagem final

# Para salvar o compósito 'l72000Composite' em variados locais

In [ ]:
# # No colab 

# # image= l72000Composite
# # task1 = ee.batch.Export.image.toDrive(
# #     image=l72000Composite,
# #     description='l72000Composite',
# #     folder='/content/drive/MyDrive/GEE_GAMLSS2000',
# #     region=per_urb,
# #     scale=30,
# #     crs='EPSG:4326'
# # )
# # task1.start()
# # display(task1.status())


# # No drive do Google
# # Defina a imagem usando clip e seleção de bandas
# image = l72000Composite.select(['B3', 'B2', 'B1']).clip(per_urb.geometry())

# visualization_params = {
#     'bands': ['B3', 'B2', 'B1'],
#     'min': 0,
#     'max': 0.3,
# }

# task2 = ee.batch.Export.image.toDrive(
#     image=image.visualize(**visualization_params),
#     description='l72000Composite Visual',
#     folder='ee_demos',
#     region=per_urb.geometry().getInfo()['coordinates'],
#     scale=30,
#     crs='EPSG:4326',
#     maxPixels=1e13,
#     fileFormat='GeoTIFF'
# )
# # Iniciar a tarefa
# task2.start()
# # Exibir status da tarefa
# display(task2.status())
# # print("Exportação iniciada. Status da tarefa:")


Essa função calcula e retorna variados índices espectrais e os adiona como bandas extras na imagem a que se aplica

In [27]:
# Função para calcular vários índices

def calcIndices(img):
    B1 = img.select('B1')
    B2 = img.select('B2')
    B3 = img.select('B3')
    B4 = img.select('B4')
    B5 = img.select('B5')
    B6 = img.select('B6')
    B7 = img.select('B7')

    # Cálculo do EVI cf. Huete et al., 2002
    evi = img.expression(
    'G * ( (nir-red) / (nir+  C1*red - C2*blue + L)  ) ',
    {
      'nir': img.select('B4'),
      'red': img.select('B3'),
      'blue': img.select('B1'),
      'G': ee.Number(2.5),
      'C1': ee.Number(6),
      'C2': ee.Number(7.5),
      'L': ee.Number(1)
    }).rename("evi")


    # Água (evi < 0)
    water = evi.lt(0).rename("water")

    # Terra (evi >= 0)
    land = evi.gte(0).rename("land")


    # Cálculo do NDVI (Normalized Difference Vegetation Index) cf. Colwell, 1974
    ndvi = img.expression(
        '(nir - red) / (nir + red)',
        {
            'red': img.select('B3'),
            'nir': img.select('B4')
        }).rename("ndvi")

    # Cálculo do NDBI (Normalized Difference Built-up Index) cf. Zha et al., 2003
    ndbi = img.expression(
        '(swir1 - nir) / (swir1 + nir)',
        {
            'swir1': img.select('B5'),
            'nir': img.select('B4')
        }).rename("ndbi")

    # Cálculo do URBI (Urban and Built-up Index) cf. Kawamura et al., 1996
    urbi = img.expression(
        '((swir2 - nir) / (swir2 + nir) + 1) * 100',
        {
            'swir2': img.select('B7'),
            'nir': img.select('B4')
        }).rename("urbi")

    # Cálculo do NDBAI (Normalized Difference Bareness Index) cf. Van Deventer et al, 1997
    ndbai = img.expression(
        '(swir1 - swir2) / (swir1 + swir2)',
        {
            'swir1': img.select('B5'),
            'swir2': img.select('B7')
        }).rename("ndbai")

    # Cálculo do BuitlUp

    step1 = land.gt(0)
    step2 = ndbi.gt(-0.15) #Threshold calculado ?
    step3 = ndbai.lte(0.15) #Threshold calculado ?
    builtup = step1.bitwiseAnd(step2).bitwiseAnd(step3).rename("builtup")

    # Cálculo do MNDWI (Modified Normalized Difference Water Index) cf.  H. Xu, 2006
    mndwi = img.expression(
        '(green - swir1) / (green + swir1)',
        {
            'green': img.select('B2'),
            'swir1': img.select('B5')
        }).rename("mndwi")

    # Cálculo do SAVI (Soil Adjusted Vegetation Index) cf. A.R. Huete, 1988
    savi = img.expression(
        '((nir - red) / (nir + red + 0.5)) * 1.5',
        {
            'red': img.select('B3'),
            'nir': img.select('B4')
        }).rename("savi")

    # Cálculo do IBI
    ibi = img.expression(
        '(2*swir1/(swir1+nir)-(nir/(nir+green)+green/(green+swir1)))/(2*swir1/(swir1+nir)+(nir/(nir+green)+green/(green+swir1)))',
        {
            'green': img.select('B2'),
            'swir1': img.select('B5'),
            'nir': img.select('B4'),
            'red': img.select('B3')
        }).rename("ibi")


    # Calcule nbui new built up index Sinha, Atumo and Verma 2016, com l=1
    nbui = img.expression(
        ' ( (swir1-nir)/(10*sqrt(swir1+tir)) ) -( ((nir-red)*(1+l)/(nir-red+1)) + ((green-swir1)/(green+swir1)))',
         {
            'nir': img.select('B4'),
            'red': img.select('B3'),
            'green': img.select('B2'),
            'swir1': img.select('B5'),
            'tir': img.select('B6'),
            'l': ee.Number(1)
        }).rename("nbui")


    return img.addBands([urbi, ndbi, ibi, savi, mndwi, ndvi, water, land, ndbai, builtup, nbui])

Aplicando a função à imagem

In [28]:
# Aplicar a função à coleção de 18 imagens do Landsat 7 sobre per_urb no período pedido

l72000Indices = calcIndices(l72000Composite)
display('Estrutura do compósito', l72000Indices.getInfo())


'Estrutura do compósito'

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B7',
   'data_type': {'type': 'PixelType',


Essas linhas permitem que se extraia os valores de cada banda/índice de uma imagem qualquer sobre pontos que se deseje amostrar e retornar um link para se baixar o arquivo CSV

In [31]:
# Extrair os valores das bandas 'urbi', 'ndbi', e 'ibi ...
URBIMediano = l72000Indices.select('urbi')

NDBIMediano = l72000Indices.select('ndbi')

IBIMediano = l72000Indices.select('ibi')

SAVIMediano = l72000Indices.select('savi')

MNDWIMediano = l72000Indices.select('mndwi')

NDVIMediano = l72000Indices.select('ndvi')

WaterMediano = l72000Indices.select('water')

LandMediano = l72000Indices.select('land')

NDBAIMediano = l72000Indices.select('ndbai')

NBUIMediano = l72000Indices.select('nbui')

BuiltUpMediano = l72000Indices.select('builtup')


rgb = l72000Composite.select(['B1', 'B2', 'B3'])





# # Selecionar uma imagem e sua banda para ter informações
# band = tempMediana.select(['temp'])
# # Verificar as propriedades da banda 'expPreditos'
# band_info = band.getInfo()
# display('Estrutura da banda escolhida (temp. mundo inteiro)', band_info)



Para se definir os valores mínimo e máximo de cada banda/índice (no perímetro de Londrina) ao se estabelecer os parâmetros de visualização em mapas

In [32]:
# Extrair os valores máximo e mínimo de alguma banda para os parâmetros de visualização
# na área delimitada do estudo

# Imagem escolhida
image = l72000Indices
# image = pred

# Banda escolhida
band_name = 'urbi'

# Escalas espaciais
# descomentar

# escala = 52.33 # 'expPreditos'
#escala = 926.625 # 'LST_Day_1km'
#escala = 500 # viir_ntl: 'b1'
#escala = 92.766 # 'population'
escala = 30 # as derivadas das bandas do Landsat



# Reduzir à região
max_pixel = image.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=per_urb,
    scale=escala,
    maxPixels=1e9
)

min_pixel = image.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=per_urb,
    scale=escala,
    maxPixels=1e9
)

# Valores max/min da banda e imagem escolhida
max_value = max_pixel.get(band_name)
min_value = min_pixel.get(band_name)
print('Minimum Pixel Value:', min_value.getInfo())
print('Maximum Pixel Value:', max_value.getInfo())




Minimum Pixel Value: 6.376005561624809
Maximum Pixel Value: 138.1300129366106


Parâmetros de visualização das diversas bandas/índices

In [33]:
indicePalette = cm.get_palette('BrBG', n_class=11)



indiceVisURBI = {
  'min': 6.37,
  'max': 138.13,
  'palette': indicePalette,
  'bands': 'urbi'
}

indiceVisIBI = {
  'min': -0.72,
  'max':  0.23,
  'palette': indicePalette,
  'bands': 'ibi'
}

indiceVisNDBI = {
  'min': -0.78,
  'max': 0.33 ,
  'palette': indicePalette,
  'bands': 'ndbi'
}


indiceVisNDBAI = {
  'min': -0.24,
  'max': 0.69 ,
  'palette': indicePalette,
  'bands': 'ndbai'
}

indiceVisBuiltUp = {
  'min': 0,
  'max': 1 ,
  'palette':  ['#996633', '#ffcc00'],
  'bands': 'builtup'
}

indiceVisWater = {
  'min': 0,
  'max': 1 ,
  'palette': ['#996633','#99ccff'],
  'bands': 'water'
}

indiceVisLand = {
  'min': 0,
  'max': 1 ,
  'palette':  ['#99ccff', '#996633'],
  'bands': 'land'
}

indiceVisNBUI = {
  'min': -0.80,
  'max': 0.54,
  'palette': indicePalette,
  'bands': 'nbui'
}




In [36]:
# Plotar as 3 camadas dos valores medianos das 18 imagens de cada índice mais as camadas da temperatura máxima média, preço estimado
# luzes noturnas e população residente estimada (100mx100m)

mapa2 = geemap.Map()
mapa2.setCenter(-51.1662,-23.3197,  12)
mapa2.addLayer(URBIMediano.clip(per_urb), indiceVisURBI , 'URBI mediano')
mapa2.addLayer(IBIMediano.clip(per_urb), indiceVisIBI , 'IBI mediano')
mapa2.addLayer(NDBIMediano.clip(per_urb), indiceVisNDBI , 'NDBI mediano')
mapa2.addLayer(NDBAIMediano.clip(per_urb), indiceVisNDBAI , 'NDBAI mediano')
mapa2.addLayer(BuiltUpMediano.clip(per_urb), indiceVisBuiltUp , 'BuiltUp mediano')
mapa2.addLayer(NBUIMediano.clip(per_urb), indiceVisNBUI , 'NBUI mediano')
mapa2.addLayerControl()

# Salvar localmente o mapa como arquivo HTML 
html_file_path = '/Users/fjcosta/Documents/projects/IndicesEspectrais/indices/mapaIndices2000.html'
mapa2.to_html(html_file_path)

mapa2


In [ ]:
# NDBI
mapa3 = geemap.Map()
mapa3.setCenter(-51.1662,-23.3197,  12)
mapa3.addLayer('basemap_selector')
mapa3.addLayer(l72000Indices.select('ndbi').clip(per_urb), indiceVisNDBI , 'NDBI mediano')
mapa3.add_colorbar_branca(colors=indicePalette, vmin=indiceVisNDBI.get('min'), vmax=indiceVisNDBI.get('max'), layer_name="NDBI mediano",caption='NDBI mediano ano 2000')

# Salvar localmente o mapa como arquivo HTML 
html_file_path = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/mapaNDBI.html'
mapa3.to_html(html_file_path)

mapa3

In [ ]:
# IBI
mapa3a = geemap.Map()
mapa3a.setCenter(-51.1662,-23.3197,  12)
mapa3a.addLayer('basemap_selector')
mapa3a.addLayer(l72000Indices.select('ibi').clip(per_urb), indiceVisIBI , 'IBI mediano')
mapa3a.add_colorbar_branca(colors=indicePalette, vmin=indiceVisIBI.get('min'), vmax=indiceVisIBI.get('max'), layer_name="IBI mediano",caption='IBI mediano ano 2000 ')
mapa3a

In [ ]:
# URBI
mapa3a = geemap.Map()
mapa3a.setCenter(-51.1662,-23.3197,  12)
mapa3a.addLayer(l72000Indices.select('urbi').clip(per_urb), indiceVisURBI , 'URBI mediano')

mapa3a.add_colorbar_branca(colors=indicePalette, vmin=indiceVisURBI.get('min'), vmax=indiceVisURBI.get('max'), layer_name="URBI mediano",caption='URBI mediano ano 2000 ')

mapa3a.addLayerControl()


# Salvar o mapa como arquivo HTML
html_file_path = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/mapaUrbiNaoClassificado.html'
mapa3a.to_html(html_file_path)


mapa3a

In [ ]:
# NBUI
mapa3a = geemap.Map()
mapa3a.setCenter(-51.1662,-23.3197,  12)
mapa3a.addLayer('basemap_selector')
mapa3a.addLayer(l72000Indices.select('nbui').clip(per_urb), indiceVisNBUI , 'NBUI mediano')
mapa3a.add_colorbar_branca(colors=indicePalette, vmin=indiceVisNBUI.get('min'), vmax=indiceVisNBUI.get('max'),
                           layer_name="NBUI mediano",caption='NBUI mediano ano 2000 ')
mapa3a.addLayerControl()
mapa3a

In [ ]:
# WATER
legenda = {
    '1 Água': '#99ccff',
    '2 Não água':'#996633'}

mapa3a = geemap.Map()
mapa3a.setCenter(-51.1662,-23.3197,  12)
mapa3a.addLayer('basemap_selector')
mapa3a.addLayer(l72000Indices.select('water').clip(per_urb), indiceVisWater  , 'Water')
mapa3a.add_legend(title="Classificação binária água/não água", legend_dict=legenda)
mapa3a.addLayerControl()
mapa3a

In [ ]:
# LAND
legenda = {
    '1 Solo': '#996633',
    '2 Não solo': '#99ccff'}

mapa3a = geemap.Map()
mapa3a.setCenter(-51.1662,-23.3197,  12)
mapa3a.addLayer('basemap_selector')
mapa3a.addLayer(l72000Indices.select('land').clip(per_urb), indiceVisLand  , 'Land')
mapa3a.add_legend(title="Classificação binária solo/não solo", legend_dict=legenda)
mapa3a.addLayerControl()
mapa3a



In [ ]:
# NDBAI
mapa3a = geemap.Map()
mapa3a.setCenter(-51.1662,-23.3197,  12)
mapa3a.addLayer('basemap_selector')
mapa3a.addLayer(l72000Indices.select('ndbai').clip(per_urb), indiceVisNDBAI , 'NDBAI')
mapa3a.add_colorbar_branca(colors=indicePalette, vmin=indiceVisNDBAI.get('min'), vmax=indiceVisNDBAI.get('max'),
                           layer_name="ndbai",caption='NDBAI mediano ano 2000 ')
mapa3a.addLayerControl()
mapa3a

In [ ]:
# BUILTUP
mapa3a = geemap.Map()
mapa3a.setCenter(-51.1662,-23.3197,  12)
mapa3a.addLayer('basemap_selector')
mapa3a.addLayer(l72000Indices.select('builtup').clip(per_urb), indiceVisBuiltUp , 'BuiltUp')
mapa3a.add_colorbar_branca(colors= ['#996633', '#ffcc00'], vmin=indiceVisBuiltUp.get('min'), vmax=indiceVisBuiltUp.get('max'),
                           layer_name="builtup",caption='BUILTUP mediano ano 2000 ')
mapa3a.addLayerControl()
mapa3a

In [ ]:
# Plotando as camdas NDBAI e BuiltUp (classificação binária)
left_layer = geemap.ee_tile_layer(l72000Indices.select('ndbai').clip(per_urb), indiceVisNDBAI, 'NDBAI mediano')
right_layer = geemap.ee_tile_layer(l72000Indices.select('builtup').clip(per_urb), indiceVisBuiltUp, 'BuiltUp mediano')

mapa8 = geemap.Map(height=500)
mapa8.centerObject(per_urb, 12)

mapa8.split_map(left_layer, right_layer)
# mapa8.add_legend(title="NBBAI (tendendo ao verde para solo e marrom para artificiais) e a BuiltUp (classificação binária)")
mapa8

In [ ]:
# Plotando as camdas WATER e LAND (imagens binárias complementaes)
left_layer = geemap.ee_tile_layer(l72000Indices.select('water').clip(per_urb), indiceVisWater, 'Water')
right_layer = geemap.ee_tile_layer(l72000Indices.select('land').clip(per_urb), indiceVisLand, 'Land')

mapa8 = geemap.Map()
mapa8.centerObject(per_urb, 12)

mapa8.split_map(left_layer, right_layer)
# mapa8.add_legend(title="Imagens binárias complementares (water/land)")
mapa8

In [ ]:
# Valores estimados
mapa4 = geemap.Map()
mapa4.setCenter(-51.1662,-23.3197,  12)
mapa4.addLayer(pred.clip(per_urb), predVis , 'Preditos')
mapa4.add_colorbar_branca(colors=predPalette,vis_params=predVis,  vmin=predVis.get('min'), vmax=predVis.get('max'), layer_name="Preços estimados", caption='Preços estimados para 2000 (R$/m2)')

# Salvar o mapa como arquivo HTML
html_file_path = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/mapaPred.html'
mapa4.to_html(html_file_path)

mapa4

In [ ]:
# Temperaturas
mapa5 = geemap.Map()
mapa5.setCenter(-51.1662,-23.3197,  12)
mapa5.addLayer(tempMediana.clip(per_urb), tempVis , 'Temperatura mediana')
mapa5.add_colorbar_branca(colors=tempPalette, vmin=tempVis.get('min'), vmax=tempVis.get('max'), layer_name="Temperatura máxima", caption='Temperatura máxima para 2000 (Celsius)')

# Salvar o mapa como arquivo HTML
html_file_path = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/mapaTemp.html'
mapa5.to_html(html_file_path)


mapa5

In [ ]:
# População
mapa6 = geemap.Map()
mapa6.setCenter(-51.1662,-23.3197,  12)
mapa6.addLayer(pop.clip(per_urb), popVis , 'População')
mapa6.add_colorbar_branca(colors=popPalette, vmin=popVis.get('min'), vmax=popVis.get('max'), layer_name="População", caption='População residente estimada (hab/ha) para 2000 ')

# Salvar o mapa como arquivo HTML
html_file_path = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/mapaPop.html'
mapa6.to_html(html_file_path)

mapa6

In [ ]:
# Luzes
mapa6 = geemap.Map()
mapa6.setCenter(-51.1662,-23.3197,  12)
mapa6.addLayer(luz.clip(per_urb) , luzVis, 'Iluminação noturna')
mapa6.add_colorbar_branca(colors=luzPalette, vmin=luzVis.get('min'), vmax=luzVis.get('max'), layer_name="Luz noturna", caption='Iluminação noturna (DN) em 2000 ')

# Salvar o mapa como arquivo HTML
html_file_path = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/mapaLuz.html'
mapa6.to_html(html_file_path)


mapa6

In [ ]:
# Criando uma única imagem com 13 bandas
# R, G, B, IBI mediano, URBI mediano, NDBI mediano, SAVI mediano, MNDWI mediano,
# NDVI mediano e temperatura máxima média, preços estimados, população e luzes noturnas  para serem usadas em classificação


rgbAll = l72000Composite.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
imagemIndicesMedianos = rgbAll.addBands([IBIMediano, URBIMediano, NDBIMediano,SAVIMediano,MNDWIMediano,NDVIMediano, WaterMediano, LandMediano, NDBAIMediano, BuiltUpMediano, NBUIMediano, tempMediana, pop, luz, pred])


display(imagemIndicesMedianos.getInfo())

print(type(imagemIndicesMedianos))

#PARA QUE 1imagemIndicesmedianosYYYY' SEJA INJETADA COMO ASSET, PRIMEIRO FAZER O UPLOAD PARA GDRIVE, DEPOIS O DOWNLOAD E ENTÃO O UPLOAD DIRETO COMO TIFF

In [ ]:
# injetar como asset 
imagemIndicesMedianosYYYY = rgbAll.addBands([IBIMediano, URBIMediano, NDBIMediano,SAVIMediano,MNDWIMediano,NDVIMediano, WaterMediano, LandMediano, NDBAIMediano, BuiltUpMediano, NBUIMediano, tempMediana, pop, luz, pred, slope, ano])


per_urb = ee.FeatureCollection('users/fjcosta/LIM_Perimetro_Urbano_V3_wsg84').geometry()
image = imagemIndicesMedianos.clip(per_urb).unmask()
image = image.toFloat()
geemap.ee_export_image_to_drive(
    image=image,
    description='teste2000',
    folder='ee_demos',
    scale=30,
    region=per_urb.getInfo()['coordinates']
)

# # Print the status of the export task
# print(task.status())



In [ ]:
# No drive do Google
# Defina a imagem usando clip e seleção de bandas
image = imagemIndicesMedianos.clip(per_urb.geometry())

visualization_params = {
    'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'IBIMediano', 'URBIMediano', 'NDBIMediano', 'SAVIMediano', 'MNDWIMediano', 'NDVIMediano', 'WaterMediano', 'LandMediano', 'NDBAIMediano', 'BuiltUpMediano', 'NBUIMediano', 'tempMediana', 'pop', 'luz', 'pred' ]
}

task2 = ee.batch.Export.image.toDrive(
    image=image.visualize(**visualization_params),
    description='teste2000',
    folder='ee_demos',
    region=per_urb.geometry().getInfo()['coordinates'],
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)
# Iniciar a tarefa
task2.start()
# Exibir status da tarefa
display(task2.status())
# print("Exportação iniciada. Status da tarefa:")

In [ ]:
import ee

# Autenticar a biblioteca cliente do Earth Engine
ee.Authenticate()
ee.Initialize()

# Definir a imagem usando clip e seleção de bandas
image = imagemIndicesMedianos.clip(per_urb.geometry())

# Definir os parâmetros de exportação
export_params = {
    'description': 'teste2000',
    'folder': 'ee_demos',
    'region': per_urb.geometry().getInfo()['coordinates'],
    'scale': 30,
    'crs': 'EPSG:4326',
    'maxPixels': 1e13,
    'fileFormat': 'GeoTIFF'
}

# Iniciar a exportação da imagem para o Google Drive
task = ee.batch.Export.image.toDrive(
    image=image,
    **export_params
)

# Iniciar a tarefa de exportação
task.start()

# Exibir status da tarefa
print("Exportação iniciada. Status da tarefa:")
print(task.status())


Extrair uma amostra de pontos nos polígonos desenhados sobre áreas reconhecidamente com e sem construção (2000) e exportar os valores medidos das bandas 'B1', 'B2', 'B3', 'B4', 'B5', 'B7' (e outras variáveis) e suas coordenadas nos pontos amostrados com/sem construção e no grid para posterior estimação (GAMLSS)  para um arquivo local (formato csv) usando 'imagemIndicesMedianos'.

Em razão da limitação (GEE) a cada consulta (5000), o total de pontos:
* do grid de estimação (GAMLSS): 88734
* dos pontos amostrais nos polígonos sem construção (poligonosSample_0): 22840
* dos pontos amostrais nos polígono com construção (poligonosSample_1: 14709

foi dividido em lotes.  

Ajustes relativos aos total de pontos e nomes de bandas são necessários.

In [ ]:
import csv
from io import StringIO

# Imagem a usar
imagem = imagemIndicesMedianos

# Extraindo pontos amostrais dentro dos polígonos
poligonosSample_0 = imagem.sampleRegions(
    collection=samplePolygon_0,
    scale=30,
    geometries=True
)

poligonosSample_1 = imagem.sampleRegions(
    collection=samplePolygon_1,
    scale=30,
    geometries=True
)


# Escolha dos pontos amostrais/polígonos
# gridSelect = grid_50 #remover landcover
# gridSelect = poligonosSample_0 #22840 #'landcover': 'Permeable',
gridSelect = poligonosSample_1 #14709 'landcover': 'Impermeable',

# Nome/caminho local onde você deseja salvar o arquivo CSV
local_path = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/dados_poligonosScale30_1.csv'

# Função para extrair dados de uma parte da grade
def extract_data_from_grid(subgrid):
    grid_sample = imagem.sampleRegions(
        collection=subgrid,
        scale=30,
        geometries=True
    )

    data = []
    for feature in grid_sample.getInfo()['features']:
        properties = feature['properties']
        coordinates = feature['geometry']['coordinates']
        ponto = {
            'lat': coordinates[1],
            'lon': coordinates[0],
            'B1': properties['B1'],
            'B2': properties['B2'],
            'B3': properties['B3'],
            'B4': properties['B4'],
            'B5': properties['B5'],
            'B7': properties['B7'],
            'pred': properties['expPreditos'],
            'pop': properties['populacao'],
            'luz': properties['luzes'],
            'ndvi': properties['ndvi'],
            'landcover': 'Impermeable'              
        }
        data.append(ponto)

    return data

# Dividir em partes menores
# num_points=22840 #sample_0
# num_points=88761 #grid_50
num_points=14709 #sample_1

# Tamanho de cada lote consultado
batch_size = 5000  # tamanho máximo
batches = [gridSelect.toList(batch_size, i) for i in range(0, num_points, batch_size)]
dados_gridSelect = []

# Iterar sobre os lotes e extrair os dados
for batch in batches:
    batch_data = extract_data_from_grid(batch)
    dados_gridSelect.extend(batch_data)

# Escrever as informações em um arquivo CSV
with open(local_path, 'w', newline='') as csvfile:
    fieldnames = ['lat', 'lon', 'B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pred', 'pop', 'luz', 'ndvi', 'landcover']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Escrever o cabeçalho
    writer.writeheader()

    # Escrever as informações
    writer.writerows(dados_gridSelect)



Carregando os dados amostrados salvos localmente

In [ ]:
# Carregar o arquivo CSV com a amostra extraída dos polígnos sem construção

dados_0 = pd.read_csv('/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/dados_poligonosScale30_0.csv')
display('Quantidade de pontos amostrais sem const.', len(dados_0))
display('Primeiro ponto amostral sem const.', dados_0.iloc[0])


dados_1 = pd.read_csv('/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/dados_poligonosScale30_1.csv')
display('Quantidade de pontos amostrais com const.', len(dados_1))
display('Primeiro ponto amostral com const.', dados_1.iloc[0])


# Tirar uma amostra aleatória de cada dataframe (são mais de 37500 pontos! )
# dados_0 = dados_0.sample(n=10000)
# dados_1 = dados_1.sample(n=10000)
# display('Quantidade de pontos amostrais sorteados sem const.', len(dados_0))
# display('Quantidade de pontos amostrais sorteados com const.', len(dados_1))


# Função para criar uma Feature do Earth Engine a partir de uma linha do DataFrame
def criar_feature(row):
    geom = ee.Geometry.Point([row['lon'], row['lat']])
    props = {key: row[key] for key in row.index if key not in ['lat', 'lon']}
    return ee.Feature(geom, props)

# Criar FeatureCollection para cada DataFrame
poligonosSample_0 = ee.FeatureCollection(dados_0.apply(criar_feature, axis=1).tolist())
poligonosSample_1 = ee.FeatureCollection(dados_1.apply(criar_feature, axis=1).tolist())

# Exibir as informações do primeiro elemento de cada FeatureCollections
display("Estrutura do primeiro ponto amostral da coleção sem const.", poligonosSample_0.first().getInfo())
display("Estrutura do primeiro ponto amostral da coleção com const.", poligonosSample_1.first().getInfo())





Polígonos estabelecidos em áreas permeáveis e uma amostra dos 22840 pontos

In [ ]:
# Exibir
# imagem = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/l72000Composite.tif'
imagem=l72000Composite


# Tirar uma amostra aleatória de cada dataframe (são mais de 37500 pontos! )
dadosSmallSample_0 = dados_0.sample(n=5000)
poligonosSmallSample_0 = ee.FeatureCollection(dadosSmallSample_0.apply(criar_feature, axis=1).tolist())


m = geemap.Map()
m.set_center(-51.17471, -23.33334, 12)
m.add_layer(imagem.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0 , 'max': 0.3} ,'Imagem visual 2000')
m.add_layer(samplePolygon_0, {'color': 'yellow'}, 'Polígonos para amostragem de áreas sem construção')
m.add_layer(poligonosSmallSample_0, {'color': 'red'}, '5 K Pontos amostrados nos polígonos de áreas sem construção (22840)')


# Salvar o mapa como arquivo HTML
html_file_path = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/mapa5KElementosAmostrais_0.html'
m.to_html(html_file_path)
m

Polígonos estabelecidos em áreas impermeáveis e uma amostra do total de 14709 pontos 

In [ ]:
# Exibir
imagem = l72000Composite

# Tirar uma amostra aleatória de cada dataframe (são mais de 14709 pontos! )
dadosSmallSample_1 = dados_1.sample(n=5000)
poligonosSmallSample_1 = ee.FeatureCollection(dadosSmallSample_1.apply(criar_feature, axis=1).tolist())


m = geemap.Map()
m.set_center(-51.17471, -23.33334, 12)
m.add_layer(imagem.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0 , 'max': 0.3} ,'Imagem visual 2000')
m.add_layer(samplePolygon_1, {'color': 'green'}, 'Polígonos para amostragem de áreas com construção')
m.add_layer(poligonosSmallSample_1, {'color': 'blue'}, '5 K Pontos amostrados nos polígonos em áreas com construção (14709)')

# Salvar o mapa como arquivo HTML
html_file_path = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/mapa5KElementosAmostrais_1.html'
m.to_html(html_file_path)
m

Pontos regularmente espaçados (grid em 50 m x 50 m ) para estimação via GAMLSS

In [ ]:
# Exibir
imagem = l72000Composite
m = geemap.Map()
m.set_center(-51.17471, -23.33334, 12)
m.add_layer(imagem.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0 , 'max': 0.3} ,'Imagem visual 2000')
m.addLayer(grid_50, {'color': 'magenta'},  'Pontos regularmente dispostos (grid de 50 m x 50 m) para estimação GAMLS', opacity=0.2 ) 


# Salvar o mapa como arquivo HTML
html_file_path = '/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/gridGAMLSS.html'
m.to_html(html_file_path)
m

In [ ]:
# # Plotando apenas as bandas RGB a partir de imagemIndicesMedianos
# mapa7 = geemap.Map()
# mapa7.setCenter(-51.1662,-23.3197,  12)
# mapa7.addLayer(imagemIndicesMedianos.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0,  'max': 0.3} , 'Bandas RGB da imagem com 11 bandas')
# mapa7

In [ ]:
# # Plotando apenas a banda expPreditos a partir de imagemIndicesMedianos
# mapa6 = geemap.Map()
# mapa6.setCenter(-51.1662,-23.3197,  12)
# mapa6.addLayer(imagemIndicesMedianos.clip(per_urb), predVis , 'Valores estimados')
# mapa6

In [ ]:
# Bandas para serem usadas na classificação (as mesmas do GAMLSS)

# prediction_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
# Ou 
prediction_bands = ['B1', 'B2', 'B3','ndbi', 'urbi','ibi', 'ndvi', 'savi', 'mndwi', 'builtup', 'water', 'land' ,'ndbai', 'nbui','temp', 'expPreditos', 'populacao', 'luzes' ]





In [ ]:
# Atribuindo uma nova feature às coleções de pontos ('landcover')

poligonosSample_0=poligonosSample_0.map(lambda feature: feature.set('landcover', 0))
poligonosSample_1=poligonosSample_1.map(lambda feature: feature.set('landcover', 1))



In [ ]:
# Combine as coleções de treinamento em uma única FeatureCollection

# trainingCollection = poligonosSample_0.merge(poligonosSample_1);


# Imprimindo as propriedades do primeiro ponto
# display("Propriedades do primeiro ponto:", trainingCollection.first().getInfo())
# display('Tamanho da coleção', trainingCollection.size().getInfo())


# Checando (dar zoom para ver os pontos)
# landcover_0 = trainingCollection.filterMetadata('landcover', 'equals', 0)
# landcover_1 = trainingCollection.filterMetadata('landcover', 'equals', 1)
# map = geemap.Map()
# map.centerObject(trainingCollection, 12)
# map.addLayer(landcover_0 , {'color': 'FF0000'}, 'Pontos amostrais com cobertura permeável')
# map.addLayer(landcover_1 , {'color': '00FF00'}, 'Pontos amostrais com cobertura impermeável')

# # # Salvar o mapa como arquivo HTML
# # html_file_path = '/content/drive/MyDrive/GEE_GAMLSS2000/mapaPontosAmostraisTreinamento.html'
# # map.to_html(html_file_path)
# map







In [ ]:
# Amostra dos pontos de treinamento
# sample = trainingCollection.randomColumn()

# training = sample.filter('random <= 0.7')
# validation = sample.filter('random > 0.3')


# training = imagemIndicesMedianos.select(prediction_bands).sampleRegions(
#     collection=trainningSample.limit(5000),
#     properties=['landcover'],
#     scale=30
# )


# validation = imagemIndicesMedianos.select(prediction_bands).sampleRegions(
#     collection=validationSample.limit(5000),
#     properties=['landcover'],
#     scale=30
# )



# Checando os pontos aleatórios de treino, validação e teste (dar zoom)
# map = geemap.Map()
# map.centerObject(trainingCollection, 12)
# map.addLayer(l72000Composite.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0 , 'max': 0.3} ,'Imagem visual 2000')
# map.addLayer(training, {'color': 'red', 'opacity': 0.3}, 'Pontos de Treinamento')
# map.addLayer(validation, {'color': 'green', 'opacity': 0.3}, 'Pontos de Validação')

# # # Salvar o mapa como arquivo HTML
# # html_file_path = '/content/drive/MyDrive/GEE_GAMLSS2000/mapaPontosTreinoValidacao.html'
# # map.to_html(html_file_path)


# map



In [ ]:
# # Amostra não supervisionada para k-médias

# trainingKmedias= imagemIndicesMedianos.select(prediction_bands).sample(
#     region= per_urb,
#     scale= 30,
#     numPixels= 5000
# )


# # Imprimindo as propriedades do primeiro ponto
# display("Propriedades do primeiro ponto:", trainingKmedias.first().getInfo())

# # Treinando o classificador Random Forest
# classifier = ee.Classifier.smileRandomForest(numberOfTrees=20).train(
#     features=training,
#     classProperty='landcover',
#     inputProperties=prediction_bands
# )

# # Treinando o classificador tree boost
# classifier2 = ee.Classifier.smileGradientTreeBoost(numberOfTrees=20).train(
#     features=training,
#     classProperty='landcover',
#     inputProperties=prediction_bands
# )


# # Treinando o classificador svm
# classifier3 = ee.Classifier.libsvm().train(
#     features=training,
#     classProperty='landcover',
#     inputProperties=prediction_bands
# )

# # Treinando o classificador cart
# classifier4 = ee.Classifier.smileCart().train(
#     features=training,
#     classProperty='landcover',
#     inputProperties=prediction_bands
# )

# # Treinando o classificador knn
# classifier5 = ee.Classifier.smileKNN(5).train(
#     features=training,
#     classProperty='landcover',
#     inputProperties=prediction_bands
# )


# # Treinando o classificador k-médias (escolhido 2 para classificação binária)
# classifier6 = ee.Clusterer.wekaKMeans(2).train(trainingKmedias)



In [ ]:
# Modelagem
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Visualização da árvore
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [ ]:
display(dados_0)
display(type(dados_0))
display(dados_1)
display(type(dados_1))

dados = dados_0.append(dados_1, ignore_index=True)
display(dados)
display(type(dados))

print("Valores ausentes em 'landcover':", dados['landcover'].isnull().sum())




In [ ]:
# Unindo os dois dataframes e convertendo os rótulos de landcover
dados['landcover'] = dados['landcover'].map({'Permeable':0,'Impermeable':1})
display('Dataframe de dados', dados)
display('Tipo de objeto', type(dados))


In [ ]:
# Extraíndo uma amostra estratificada de 30% 'dados' 11227 elementos
dadosSample = dados.groupby('landcover', group_keys=False).apply(lambda x: x.sample(frac=0.3, random_state=1963))
display('Amostra de dados', dadosSample)
display('Tipo de objeto', type(dadosSample))

# Salvando para usar com GAMLSS
dadosSample.to_csv('/home/fjcosta/Documentos/Base de Dados/Base de Dados Londrina/Dados espectrais/dadosSample.csv', index=False)

In [ ]:
# Separando em um dataframe de features e um de respostas (X e y)
X = dadosSample.drop(['lat', 'lon', 'landcover'], axis=1)
y = dadosSample['landcover']

display("\nValores ausentes após remover 'landcover' de X:", X.isnull().sum())
display("\nValores ausentes após remover 'landcover' de y:", y.isnull().sum())

display("X",X)
display("y", y)

# Executar a divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1963)

# Examinar a distribuição de 'landcover' em y_train após a divisão
print("\nDistribuição de 'landcover' em y_train após a divisão:")
print(y_train.value_counts())

# Examinar a distribuição de 'landcover' em y_test após a divisão
print("\nDistribuição de 'landcover' em y_test após a divisão:")
print(y_test.value_counts())



display('X_train', X_train)
display('X_test', X_test)
display('y_train', y_train)
display('y_test', y_test)

X.describe()
y.describe()



In [ ]:
print('Training Features Shape:', X_train.shape)
print('Training label Shape:', y_train.shape)

print('Testing Features Shape:', X_test.shape)
print('Testing labels Shape:', y_test.shape)


In [ ]:
# Import the model we are using
# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# rf.fit(X_train, y_train);


rf = RandomForestClassifier()
rf.fit(X_train, y_train)


In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:

from sklearn.tree import export_graphviz
import os
import graphviz
from sklearn.ensemble import RandomForestClassifier
# # Export the first three decision trees from the forest

for i in range(3):
    tree = rf.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=X_train.columns,  
                               filled=True,  
                               max_depth=2, 
                               impurity=False, 
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)


In [ ]:
# Generate predictions with the best model
y_pred = rf.predict(X_test)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred)

ConfusionMatrixDisplay(confusion_matrix=cm).plot();

In [ ]:
# Create a series containing feature importances from the model and feature names from the training data
feature_importances = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

# Plot a simple bar chart
feature_importances.plot.bar();

In [ ]:
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)

In [ ]:
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

# Generate predictions with the best model
y_pred = best_rf.predict(X_test)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred)

ConfusionMatrixDisplay(confusion_matrix=cm).plot();




In [ ]:
# Create a series containing feature importances from the model and feature names from the training data
feature_importances = pd.Series(best_rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

# Plot a simple bar chart
feature_importances.plot.bar();

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X, y)

trees = ml.rf_to_strings(clf, feature_names)

In [ ]:
# acuraciaTreino = classifier.confusionMatrix()
# display('Confusion matrix no treinamento RF', acuraciaTreino)
# display('Acurácia geral no treinamento RF', acuraciaTreino.accuracy())

# acuraciaTreino2 = classifier2.confusionMatrix()
# display('Confusion matrix no treinamento GTB', acuraciaTreino2)
# display('Acurácia geral no treinamento GTB', acuraciaTreino2.accuracy())

# acuraciaTreino3 = classifier3.confusionMatrix()
# display('Confusion matrix no treinamento SVM', acuraciaTreino3)
# display('Acurácia geral no treinamento SVM', acuraciaTreino3.accuracy())

# acuraciaTreino4 = classifier4.confusionMatrix()
# display('Confusion matrix no treinamento CART', acuraciaTreino4)
# display('Acurácia geral no treinamento CART', acuraciaTreino4.accuracy())

# acuraciaTreino5 = classifier5.confusionMatrix()
# display('Confusion matrix no treinamento knn', acuraciaTreino5)
# display('Acurácia geral no treinamento knn', acuraciaTreino5.accuracy())




In [ ]:
validation = validation.classify(classifier)
acuraciaValidacao = validation.errorMatrix('landcover', 'classification')
display('Confusion matrix validação RF', acuraciaValidacao )
display('Acurácia na validação RF', acuraciaValidacao.accuracy())


validation2 = validation.classify(classifier2)
acuraciaValidacao2 = validation2.errorMatrix('landcover', 'classification')
display('Confusion matrix validação GTB', acuraciaValidacao2 )
display('Acurácia na validação GTB', acuraciaValidacao2.accuracy())

validation3 = validation.classify(classifier3)
acuraciaValidacao3 = validation3.errorMatrix('landcover', 'classification')
display('Confusion matrix validação SVM', acuraciaValidacao3 )
display('Acurácia na validação SVM', acuraciaValidacao3.accuracy())


validation4 = validation.classify(classifier4)
acuraciaValidacao4 = validation4.errorMatrix('landcover', 'classification')
display('Confusion matrix validação CART', acuraciaValidacao4 )
display('Acurácia na validação CART', acuraciaValidacao4.accuracy())


validation5 = validation.classify(classifier5)
acuraciaValidacao5 = validation5.errorMatrix('landcover', 'classification')
display('Confusion matrix validação knnT', acuraciaValidacao5 )
display('Acurácia na validação knn', acuraciaValidacao5.accuracy())


In [ ]:
display('Resultados do classificador treinado RF', classifier.explain())

varImportancia = ee.Feature(None, classifier.explain().get('importance'))

importancias= [varImportancia.get('B1').getInfo(),varImportancia.get('B2').getInfo(),varImportancia.get('B3').getInfo(),
               varImportancia.get('B4').getInfo(),varImportancia.get('B5').getInfo(),varImportancia.get('B7').getInfo()]



display('Resultados do classificador treinado GTB', classifier2.explain())

varImportancia2 = ee.Feature(None, classifier2.explain().get('importance'))

importancias2= [varImportancia2.get('B1').getInfo(),varImportancia2.get('B2').getInfo(),varImportancia2.get('B3').getInfo(),
                varImportancia2.get('B4').getInfo(),varImportancia2.get('B5').getInfo(),varImportancia2.get('B7').getInfo()]


display('Resultados do classificador treinado CART', classifier4.explain())

varImportancia4 = ee.Feature(None, classifier4.explain().get('importance'))

importancias4= [varImportancia4.get('B1').getInfo(),varImportancia4.get('B2').getInfo(),varImportancia4.get('B3').getInfo(),
                varImportancia4.get('B4').getInfo(),varImportancia4.get('B5').getInfo(),varImportancia4.get('B7').getInfo()]


bandas = ['Blue','Green','Red','NIR','SWIR1','SWIR2']


In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.bar(bandas, importancias)
ax.set_title('RF Importância das bandas ')
ax.set_xlabel('Bandas')
ax.set_ylabel('Importancias')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.bar(bandas, importancias2)
ax.set_title('GTB Importância das bandas ')
ax.set_xlabel('Bandas')
ax.set_ylabel('Importancias')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.bar(bandas, importancias4)
ax.set_title('CART Importância das bandas ')
ax.set_xlabel('Bandas')
ax.set_ylabel('Importancias')
plt.show()

In [ ]:
# Classificar a imagem

classified =  imagemIndicesMedianos.select(prediction_bands).classify(classifier)
classified2 = imagemIndicesMedianos.select(prediction_bands).classify(classifier2)
classified3 = imagemIndicesMedianos.select(prediction_bands).classify(classifier3)
classified4 = imagemIndicesMedianos.select(prediction_bands).classify(classifier4)
classified5 = imagemIndicesMedianos.select(prediction_bands).classify(classifier5)
classified6 = imagemIndicesMedianos.cluster(classifier6);



In [ ]:
# Obtenha todos os valores únicos da propriedade 'landcover' dos dados de treinamento
unique_classes = training.distinct('landcover').aggregate_array('landcover')

# Converte a lista de classes para um objeto Python
unique_classes_py = unique_classes.getInfo()

# Imprima a ordem das classes
print('Ordem das classes:', unique_classes_py)



In [ ]:
# Classificações binárias apenas

pal1=(['#996633','#ffcc00'])

clasVis = {
    'min': 0,
    'max': 1,
    'palette': pal1
}

# Classificação binária apenas

legenda = {
    '1 Solo permeável': '#996633',
    '2 Solo impermeável': '#ffcc00'}



In [ ]:
# Calculando a área de cada classe no perímetro urbano de Londrina para o classificador Random Forest

area_classified = ee.Image.pixelArea().divide(1e6).addBands(classified).reduceRegion(
    reducer=ee.Reducer.sum().group(groupField=1, groupName='group'),
    geometry=per_urb,
    scale=30,
    maxPixels=10e10
)

# Crie um dicionário para mapear rótulos aos nomes correspondentes das classes

label_mapping = {
    '0': 'Permeável',
    '1': 'Não permeável'
}

outputReducers = ee.List(area_classified.get('groups'))

display('Grupos da classificação' ,  outputReducers.getInfo())
display('Área classificada como 0', outputReducers.getInfo()[0]['sum'])
display('Área classificada como 1', outputReducers.getInfo()[1]['sum'])


# Itere sobre os resultados e imprima os rótulos associados aos valores
for reducer in outputReducers.getInfo():
    label = label_mapping.get(str(reducer['group']), 'Desconhecido')
    area = reducer['sum']
    print(f'Área de {label}: {area} km2')

display('Áreas das classes:', outputReducers.getInfo())

display('Área urbana de Londrina:', per_urb.geometry().area().divide(1e6).getInfo())

# Obtenha a lista de nomes das bandas (classes)

class_band_names = classified.bandNames()

# Converte a lista para um objeto Python

class_band_names_py = class_band_names.getInfo()

text = ('Área permeável: ' + str(round(outputReducers.getInfo()[0]['sum'], 2)) + ' km² <BR> \n'
        'Área não permeável: ' + str(round(outputReducers.getInfo()[1]['sum'], 2)) + ' km² <BR> \n'
        'Área total do per. urbano de Londrina: '  + str(round  (per_urb.geometry().area().divide(1e6).getInfo())) + ' km² (2000)'
        )

params = {
    'fontsize': 18,
    'fontcolor': 'black',
    'bold': False,
    'padding': '10px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '5px',
    'position': 'bottomleft',
}


In [ ]:
# Plotando uma imagem visual e a classificação com RF
# Camadas num painel subdividido
# Para salvar o html com 'split'

import folium
import geemap.foliumap as geemap # a poder salvar maps 'splited'


visual = imagemIndicesMedianos
classificada = classified

left_layer = geemap.ee_tile_layer(visual.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3}, 'Imagem a classificar')
right_layer = geemap.ee_tile_layer(classificada.clip(per_urb),  clasVis, 'Imagem classificada com RF')

mapa8 = geemap.Map()
# mapa8.addLayer('basemap_selector') #Usand foliun não pode usar
mapa8.centerObject(per_urb, 12)

mapa8.split_map(left_layer, right_layer)
mapa8.add_legend(title="Classificação para o ano de 2000 via RF", legend_dict=legenda)
mapa8.add_text(text, **params)



# Salvar o mapa como arquivo HTML
html_file_path = '/content/drive/MyDrive/GEE_GAMLSS2000/mapaRFclassificada.html'
mapa8.to_html(html_file_path)


mapa8


In [ ]:
# Plotando uma imagem visual e a classificação com GTB
# Camadas num painel subdividido

visual = imagemIndicesMedianos
classificada = classified2

left_layer = geemap.ee_tile_layer(visual.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0 ,'max': 0.3}, 'Imagem a classificar')
right_layer = geemap.ee_tile_layer(classificada.clip(per_urb), clasVis, 'Imagem classificada com GTB')

mapa9 = geemap.Map()
mapa9.centerObject(per_urb, 12)

mapa9.split_map(left_layer, right_layer)
mapa9.add_legend(title="Classificação para o ano de 2000 via GTB", legend_dict=legenda)
mapa9.addLayerControl()
mapa9

In [ ]:
# Calculando a área de cada classe no perímetro urbano de Londrina para o classificador SVM

area_classified = ee.Image.pixelArea().divide(1e6).addBands(classified3).reduceRegion(
    reducer=ee.Reducer.sum().group(groupField=1, groupName='group'),
    geometry=per_urb,
    scale=30,
    maxPixels=10e10
)

# Crie um dicionário para mapear rótulos aos nomes correspondentes das classes

label_mapping = {
    '0': 'Permeável',
    '1': 'Não permeável'
}

outputReducers = ee.List(area_classified.get('groups'))

pprint(outputReducers.getInfo())

print(outputReducers.getInfo()[0]['sum'])

# Itere sobre os resultados e imprima os rótulos associados aos valores

for reducer in outputReducers.getInfo():
    label = label_mapping.get(str(reducer['group']), 'Desconhecido')
    area = reducer['sum']
    print(f'Área de {label}: {area} km2')

print('Áreas das classes:', outputReducers.getInfo())

print('Área urbana de Londrina:', per_urb.geometry().area().divide(1e6).getInfo())

# Obtenha a lista de nomes das bandas (classes)

class_band_names = classified.bandNames()

# Converte a lista para um objeto Python

class_band_names_py = class_band_names.getInfo()

text = ('Área permeável: ' + str(round(outputReducers.getInfo()[0]['sum'], 2)) + ' km² <BR> \n'
        'Área não permeável: ' + str(round(outputReducers.getInfo()[1]['sum'], 2)) + ' km² <BR> \n'
        'Área total do per. urbano de Londrina: '  + str(round  (per_urb.geometry().area().divide(1e6).getInfo())) + ' km² (2000)'
        )
params = {
    'fontsize': 18,
    'fontcolor': 'black',
    'bold': False,
    'padding': '10px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '5px',
    'position': 'bottomleft',
}


In [ ]:
# Plotando uma imagem visual e a classificação com SVM
# Camadas num painel subdividido

visual = imagemIndicesMedianos
classificada = classified3

left_layer = geemap.ee_tile_layer(visual.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0,  'max': 0.3}, 'Imagem a classificar')
right_layer = geemap.ee_tile_layer(classificada.clip(per_urb),clasVis, 'Imagem classificada com SVM')

mapa10 = geemap.Map()
mapa10.centerObject(per_urb, 12)

mapa10.split_map(left_layer, right_layer)
mapa10.add_legend(title="Classificação para o ano de 2000 via SVM", legend_dict=legenda)
mapa10.add_text(text, **params)
mapa10

In [ ]:
# Calculando a área de cada classe no perímetro urbano de Londrina para o classificador CART

area_classified = ee.Image.pixelArea().divide(1e6).addBands(classified4).reduceRegion(
    reducer=ee.Reducer.sum().group(groupField=1, groupName='group'),
    geometry=per_urb,
    scale=30,
    maxPixels=10e10
)

# Crie um dicionário para mapear rótulos aos nomes correspondentes das classes

label_mapping = {
    '0': 'Permeável',
    '1': 'Não permeável'
}

outputReducers = ee.List(area_classified.get('groups'))

pprint(outputReducers.getInfo())

print(outputReducers.getInfo()[0]['sum'])

# Itere sobre os resultados e imprima os rótulos associados aos valores

for reducer in outputReducers.getInfo():
    label = label_mapping.get(str(reducer['group']), 'Desconhecido')
    area = reducer['sum']
    print(f'Área de {label}: {area} km2')

print('Áreas das classes:', outputReducers.getInfo())

print('Área urbana de Londrina:', per_urb.geometry().area().divide(1e6).getInfo())

# Obtenha a lista de nomes das bandas (classes)

class_band_names = classified.bandNames()

# Converte a lista para um objeto Python

class_band_names_py = class_band_names.getInfo()

text = ('Área permeável: ' + str(round(outputReducers.getInfo()[0]['sum'], 2)) + ' km² <BR> \n'
        'Área não permeável: ' + str(round(outputReducers.getInfo()[1]['sum'], 2)) + ' km² <BR> \n'
        'Área total do per. urbano de Londrina: '  + str(round  (per_urb.geometry().area().divide(1e6).getInfo())) + ' km² (2000)'
        )
params = {
    'fontsize': 18,
    'fontcolor': 'black',
    'bold': False,
    'padding': '10px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '5px',
    'position': 'bottomleft',
}


In [ ]:
# Plotando uma imagem visual e a classificação com CART
# Camadas num painel subdividido

visual = imagemIndicesMedianos
classificada = classified4

left_layer = geemap.ee_tile_layer(visual.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3}, 'Imagem a classificar')
right_layer = geemap.ee_tile_layer(classificada.clip(per_urb),clasVis, 'Imagem classificada com CART')

mapa11 = geemap.Map()
mapa11.centerObject(per_urb, 12)

mapa11.split_map(left_layer, right_layer)
mapa11.add_legend(title="Classificação para o ano de 2000 via CART", legend_dict=legenda)
mapa11.add_text(text, **params)
mapa11

In [ ]:
# Calculando a área de cada classe no perímetro urbano de Londrina para o classificador KNN

area_classified = ee.Image.pixelArea().divide(1e6).addBands(classified5).reduceRegion(
    reducer=ee.Reducer.sum().group(groupField=1, groupName='group'),
    geometry=per_urb,
    scale=30,
    maxPixels=10e10
)

# Crie um dicionário para mapear rótulos aos nomes correspondentes das classes

label_mapping = {
    '0': 'Permeável',
    '1': 'Não permeável'
}

outputReducers = ee.List(area_classified.get('groups'))

pprint(outputReducers.getInfo())

print(outputReducers.getInfo()[0]['sum'])

# Itere sobre os resultados e imprima os rótulos associados aos valores

for reducer in outputReducers.getInfo():
    label = label_mapping.get(str(reducer['group']), 'Desconhecido')
    area = reducer['sum']
    print(f'Área de {label}: {area} km2')

print('Áreas das classes:', outputReducers.getInfo())

print('Área urbana de Londrina:', per_urb.geometry().area().divide(1e6).getInfo())

# Obtenha a lista de nomes das bandas (classes)

class_band_names = classified.bandNames()

# Converte a lista para um objeto Python

class_band_names_py = class_band_names.getInfo()

text = ('Área permeável: ' + str(round(outputReducers.getInfo()[0]['sum'], 2)) + ' km² <BR> \n'
        'Área não permeável: ' + str(round(outputReducers.getInfo()[1]['sum'], 2)) + ' km² <BR> \n'
        'Área total do per. urbano de Londrina: '  + str(round  (per_urb.geometry().area().divide(1e6).getInfo())) + ' km² (2000)'
        )
params = {
    'fontsize': 18,
    'fontcolor': 'black',
    'bold': False,
    'padding': '10px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '5px',
    'position': 'bottomleft',
}


In [ ]:
# Plotando uma imagem visual e a classificação com KNN
# Camadas num painel subdividido

visual = imagemIndicesMedianos
classificada = classified5

left_layer = geemap.ee_tile_layer(visual.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0,'max': 0.3}, 'Imagem a classificar')
right_layer = geemap.ee_tile_layer(classificada.clip(per_urb), clasVis, 'Imagem classificada com KNN')

mapa11 = geemap.Map()
mapa11.centerObject(per_urb, 12)

mapa11.split_map(left_layer, right_layer)
mapa11.add_legend(title="Classificação para o ano de 2000 via KNN", legend_dict=legenda)
mapa11.add_text(text, **params)
mapa11



In [ ]:
# Calculando a área de cada classe no perímetro urbano de Londrina para o classificador KMÉDIAS

area_classified = ee.Image.pixelArea().divide(1e6).addBands(classified6).reduceRegion(
    reducer=ee.Reducer.sum().group(groupField=1, groupName='group'),
    geometry=per_urb,
    scale=30,
    maxPixels=10e10
)

# Obtendo os resultados da região reduzida
resultados_regiao = area_classified.getInfo()

# Extraindo informações sobre as classes '0' e '1'
classe_0 = resultados_regiao['groups'][0]['sum']
classe_1 = resultados_regiao['groups'][1]['sum']

# Imprimindo as contagens das classes
print('Contagem da Classe 0:', classe_0) #impermeável
print('Contagem da Classe 1:', classe_1) #permeável



# Crie um dicionário para mapear rótulos aos nomes correspondentes das classes

label_mapping = {
    '0': 'Impermeável',
    '1': 'Permeável'
}

outputReducers = ee.List(area_classified.get('groups'))

pprint(outputReducers.getInfo())

print(outputReducers.getInfo()[0]['sum'])

# Itere sobre os resultados e imprima os rótulos associados aos valores

for reducer in outputReducers.getInfo():
    label = label_mapping.get(str(reducer['group']), 'Desconhecido')
    area = reducer['sum']
    print(f'Área de {label}: {area} km2')

print('Áreas das classes:', outputReducers.getInfo())

print('Área urbana de Londrina:', per_urb.geometry().area().divide(1e6).getInfo())

# Obtenha a lista de nomes das bandas (classes)

class_band_names = classified.bandNames()

# Converte a lista para um objeto Python

class_band_names_py = class_band_names.getInfo()

text = ('Área impermeável: ' + str(round(outputReducers.getInfo()[0]['sum'], 2)) + ' km² <BR> \n'
        'Área permeável: ' + str(round(outputReducers.getInfo()[1]['sum'], 2)) + ' km² <BR> \n'
        'Área total do per. urbano de Londrina: '  + str(round  (per_urb.geometry().area().divide(1e6).getInfo())) + ' km² (2000)'
        )

params = {
    'fontsize': 18,
    'fontcolor': 'black',
    'bold': False,
    'padding': '10px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '5px',
    'position': 'bottomleft',
}

# Classificações binárias apenas

pal1=(['#ffcc00','#996633'])

clasVis = {
    'min': 0,
    'max': 1,
    'palette': pal1
}

# Classificação binária apenas


legenda = {
    '1 Solo permeável': '#996633',
    '2 Solo impermeável': '#ffcc00'}



In [ ]:
# Plotando uma imagem visual e a classificação com k-Médias
# Camadas num painel subdividido

visual = imagemIndicesMedianos
classificada = classified6

left_layer = geemap.ee_tile_layer(visual.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0,  'max': 0.3}, 'Imagem a classificar')
right_layer = geemap.ee_tile_layer(classificada.clip(per_urb), clasVis, 'Imagem classificada com k-Médias')

mapa9 = geemap.Map()
mapa9.centerObject(per_urb, 12)
mapa9.split_map(left_layer, right_layer)
mapa9.add_legend(title="Classificação para o ano de 2000 via Kmédias", legend_dict=legenda)
mapa9.add_text(text, **params)
mapa9

In [ ]:
# Limiares de Otsu (classificação binária)
# Histograma da banda urbi
histURBI = URBIMediano.select('urbi').reduceRegion(
    **{
        'reducer': ee.Reducer.histogram(255, 2),
        'geometry': per_urb,
        'scale': 10,
        'bestEffort': True,
    }
)

# Histograma da banda urbi
histNDBI = NDBIMediano.select('ndbi').reduceRegion(
    **{
        'reducer': ee.Reducer.histogram(255, 2),
        'geometry': per_urb,
        'scale': 10,
        'bestEffort': True,
    }
)

# Histograma da banda urbi
histIBI = IBIMediano.select('ibi').reduceRegion(
    **{
        'reducer': ee.Reducer.histogram(255, 2),
        'geometry': per_urb,
        'scale': 10,
        'bestEffort': True,
    }
)


# Histograma da banda ndbai
histNDBAI = NDBAIMediano.select('ndbai').reduceRegion(
    **{
        'reducer': ee.Reducer.histogram(255, 2),
        'geometry': per_urb,
        'scale': 10,
        'bestEffort': True,
    }
)

# Histograma da banda ndbai
histNBUI = NBUIMediano.select('nbui').reduceRegion(
    **{
        'reducer': ee.Reducer.histogram(255, 2),
        'geometry': per_urb,
        'scale': 10,
        'bestEffort': True,
    }
)






In [ ]:
# Plotando
# hist_dict = histURBI.getInfo()
# print(hist_dict)
# x = hist_dict['urbi']['bucketMeans']
# y = hist_dict['urbi']['histogram']
# plt.bar(x, y)
# plt.show()

In [ ]:
# Função para extrair o valor qe maximiza a variância entre classes sucessivamente estabelecidas nos bins de subdivisão

def otsu(histogram):
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sum.divide(total)

    indices = ee.List.sequence(1, size)

    # Compute between sum of squares, where each mean partitions the data.

    def func_xxx(i):
        aCounts = counts.slice(0, 0, i)
        aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
        aMeans = means.slice(0, 0, i)
        aMean = (
            aMeans.multiply(aCounts)
            .reduce(ee.Reducer.sum(), [0])
            .get([0])
            .divide(aCount)
        )
        bCount = total.subtract(aCount)
        bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
        return aCount.multiply(aMean.subtract(mean).pow(2)).add(
            bCount.multiply(bMean.subtract(mean).pow(2))
        )

    bss = indices.map(func_xxx)

    # Return the mean value corresponding to the maximum BSS.
    return means.sort(bss).get([-1])

In [ ]:
# # Ver também em https://gis.stackexchange.com/questions/351740/multi-otsu-algorithm-in-google-earth-engine

# def computeThresholdUsingOtsutri(image,scale,zone):
#     bounds=zone
#     minValue=-0.4
#     imageEdge= image.clip(zone)
#     buckets = 50
#     hist = ee.Dictionary(imageEdge.reduceRegion(ee.Reducer.histogram(buckets), bounds, scale,bestEffort=True,maxPixels=1000000000));
#     threshold=ee.Algorithms.If(hist.contains('bucketMeans'), otsu_trimodal(hist), minValue)
#     threshold=ee.List(threshold)
#     threshold_k = ee.Number(threshold.get(0))
#     threshold_i=ee.Number(threshold.get(1))
#     print(threshold.get(0).getInfo())
#     print(threshold.get(1).getInfo())
#     output=image.addBands(
#     image.gt(threshold_k.min(threshold_i)).And(image).lt(threshold_k.max(threshold_i))).selfMask().rename("water").clip(zone)
# )

#     return output

# def otsu_trimodal(histogram):
#     histogram = ee.Dictionary(histogram)

#     counts = ee.Array(histogram.get('histogram'));
#     means = ee.Array(histogram.get('bucketMeans'));
#     size = means.length().get([0]);
#     total = counts.reduce(ee.Reducer.sum(), [0]).get([0]);
#     sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0]);
#     mean = sum.divide(total);
#     #indices = ee.List.sequence(1, size);

#     def f_(i):
#         global k
#         aCounts = counts.slice(0, 0, i)
#         aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0]);
#         aMeans = means.slice(0, 0, i)
#         aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCount)

#         bCounts=counts.slice(0,-1,k)
#         bCount=bCounts.reduce(ee.Reducer.sum(), [0]).get([0])
#         bMeans=means.slice(0,-1,k)
#         bMean=bMeans.multiply(bCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(bCount)

#         #cCount=total.subtract(aCount).subtract(bCount)
#         cCounts=counts.slice(0,i,k)
#         cCount=cCounts.reduce(ee.Reducer.sum(), [0]).get([0])
#         cMeans=means.slice(0,i,k)
#         cMean=cMeans.multiply(cCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(cCount)

#         return aCount.multiply(aMean.subtract(mean).pow(2)).add(bCount.multiply(bMean.subtract(mean).pow(2))).add(cCount.multiply(cMean.subtract(mean).pow(2)))


#     global bss_
#     global k
#     bss_=ee.List([[0]])
#     k=ee.Number(-1)
#     #while (k.getInfo())>= (indices.size().multiply(-1).getInfo()):
#     while (k.getInfo())> (size.multiply(-1).getInfo()):
#         indices = ee.List.sequence(1, ee.Number(size).add(k))
#         intervariance=ee.List(indices.map(f_))
#         intervar_=ee.Algorithms.If(intervariance.size().lt(size),intervariance.cat(ee.List.repeat(0,size.subtract(intervariance.size()))),intervariance)
#         bss_=bss_.cat([intervar_])
#         k=k.subtract(1)
#     bss_=bss_.slice(1,bss_.size())
#     bss_=ee.Array(bss_)
#     k_max=ee.Number(bss_.argmax().get(0)).add(1).multiply(-1) #### El argmax que no sean dos valores practicamente iguales
#     i_max=ee.Number(bss_.argmax().get(1))

#     return [means.get([k_max]),means.get([i_max])]


In [ ]:
thresholdURBI = otsu(histURBI.get('urbi'))
display('Limiar de Otsu (threshold) para urbi', thresholdURBI.getInfo())

thresholdNDBI = otsu(histNDBI.get('ndbi'))
display('Limiar de Otsu (threshold) para ndbi', thresholdNDBI.getInfo())

thresholdIBI = otsu(histIBI.get('ibi'))
display('Limiar de Otsu (threshold) para ibi', thresholdIBI.getInfo())

thresholdNDBAI = otsu(histNDBAI.get('ndbai'))
display('Limiar de Otsu (threshold) para ndbai', thresholdNDBAI.getInfo())

thresholdNBUI = otsu(histNBUI.get('nbui'))
display('Limiar de Otsu (threshold) para nbui', thresholdNBUI.getInfo())



In [ ]:
# paletas de cores para as imagens de classificação binária

indiceBinPaletteR = ["ffff99","#bf8040"]
indiceBinPalette = ["#bf8040","#ffff99"]

In [ ]:
# Extraído áreas
# per_munArea = per_mun.geometry().area()
# per_munAreaSqKm = ee.Number(per_munArea).divide(1e6).round()
# print('Area total do município (IBGE: 1652,569  km²)', per_munAreaSqKm.getInfo())
# perímetro rubano (2019): 129,44 km²

per_urbArea = per_urb.geometry().area()
per_urbAreaSqKm = ee.Number(per_urbArea).divide(1e6).round()
display('Area do perímetro urbano do município: ', per_urbAreaSqKm.getInfo())

area_urbana = per_urbAreaSqKm.getInfo()

per_munArea = per_mun.geometry().area()
per_munAreaSqKm = ee.Number(per_munArea).divide(1e6).round()
display('Area do município ', per_munAreaSqKm.getInfo())

In [ ]:
# Quantificação URBI
imagemURBI =  l72000Indices.select('urbi')

pixel_area = ee.Image.pixelArea().reproject('EPSG:4326', None,  30)

URBI0_area = pixel_area.updateMask(imagemURBI.lt(thresholdURBI))
URBI1_area = pixel_area.updateMask(imagemURBI.gt(thresholdURBI))

area0 = URBI0_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=per_urb,
    crs= "EPSG: 4326",
    scale= 30,
    maxPixels=1e9,
)

area1 = URBI1_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=per_urb,
    crs= "EPSG: 4326",
    scale= 30,
    maxPixels=1e9,
)


square_meters0 = area0.getNumber('area').getInfo()
square_kilometers0 = square_meters0 / 1e6
display('Área (km²) com URBI abaixo do limiar', round(square_kilometers0,2))

square_meters1 = area1.getNumber('area').getInfo()
square_kilometers1 = square_meters1 / 1e6
display('Área (km²) com URBI acima do limiar', round(square_kilometers1,2))


In [ ]:
# Classificação binária (Otsu threshold) pelo URBI
# Para salvar o html com 'split'

import folium
import geemap.foliumap as geemap # a poder salvar maps 'splited'

BuiltUp_urbi = URBIMediano.select('urbi').gt(thresholdURBI)

visual = imagemIndicesMedianos

text = 'Área impermeável (manmade) cf. URBI: ' + str( round(square_kilometers1,2) )+ ' km² (2000) <BR> Área permeável cf. URBI: ' + str( round(square_kilometers0,2) )+ ' km² (2000) <BR> Área urbana cf. IPPUL: ' + str(area_urbana)+ ' km² (2019)'

params = {
    'fontsize': 18,
    'fontcolor': 'black',
    'bold': False,
    'padding': '10px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '5px',
    'position': 'bottomleft',
}

map = geemap.Map()
map.centerObject(per_urb, 12)
left_layer = geemap.ee_tile_layer(BuiltUp_urbi.clip(per_urb), {'palette': indiceBinPalette}, 'Cobertura artificial (manmade) cf. urbi')
right_layer = geemap.ee_tile_layer(l72000Composite.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0 , 'max': 0.3} ,'Imagem visual 2000')
map.split_map(left_layer, right_layer)
map.add_text(text, **params)

# Salvar o mapa como arquivo HTML
html_file_path = '/content/drive/MyDrive/GEE_GAMLSS2000/mapaURBIclassificado.html'
map.to_html(html_file_path)
map




In [ ]:
#  Classificação binária (Otsu threshold) pelo URBI num painel subdividido
BuiltUp_urbi = URBIMediano.select('urbi').gt(thresholdURBI)
NonBuiltUp_urbi = URBIMediano.select('urbi').lt(thresholdURBI)
visual = imagemIndicesMedianos

left_layer = geemap.ee_tile_layer(BuiltUp_urbi.mask(BuiltUp_urbi).clip(per_urb), {'palette': indiceBinPalette}, 'Cobertura artificial (manmade) cf. urbi')
right_layer = geemap.ee_tile_layer(NonBuiltUp_urbi.mask(NonBuiltUp_urbi).clip(per_urb), {'palette': indiceBinPaletteR}, 'Cobertura natural cf. urbi')

text = 'Área impermeável (manmade) cf. URBI: ' + str( round(square_kilometers1,2) )+ ' km² (2000) <BR> Área permeável cf. URBI: ' + str( round(square_kilometers0,2) )+ ' km² (2000) <BR> Área urbana cf. IPPUL: ' + str(area_urbana)+ ' km² (2019)'
params = {
    'fontsize': 18,
    'fontcolor': 'black',
    'bold': False,
    'padding': '10px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '5px',
    'position': 'bottomleft',
}
mapa11 = geemap.Map()
mapa11.centerObject(per_urb, 12)
mapa11.split_map(left_layer, right_layer)
mapa11.add_text(text, **params)
mapa11.addLayer(visual.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3}, 'Imagem a classificar')
mapa11



In [ ]:
# Quantificação NBUI
imagemNBUI =  l72000Indices.select('nbui')

pixel_area0 = ee.Image.pixelArea().reproject('EPSG:4326', None,  30)
NBUI0_area = pixel_area0.updateMask(imagemNBUI.lt(thresholdNBUI))
area0 = NBUI0_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=per_urb,
    crs= "EPSG: 4326",
    scale= 30,
    maxPixels=1e9,
)
square_meters0 = area0.getNumber('area').getInfo()
square_kilometers0 = square_meters0 / 1e6
display('Área (km²) com NBUI abaixo do limiar', round(square_kilometers0,2))


pixel_area1 = ee.Image.pixelArea().reproject('EPSG:4326', None,  30)
NBUI1_area = pixel_area1.updateMask(imagemNBUI.gt(thresholdNBUI))
area1 = NBUI1_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=per_urb,
    crs= "EPSG: 4326",
    scale= 30,
    maxPixels=1e9,
)
square_meters1 = area1.getNumber('area').getInfo()
square_kilometers1 = square_meters1 / 1e6
display('Área (km²) com NBUI acima do limiar', round(square_kilometers1,2))



In [ ]:
#  Classificação binária (Otsu threshold) pelo NBUI num painel subdividido
BuiltUp_nbui = NBUIMediano.select('nbui').gt(thresholdNBUI)
NonBuiltUp_nbui = NBUIMediano.select('nbui').lt(thresholdNBUI)
visual = imagemIndicesMedianos

left_layer = geemap.ee_tile_layer(BuiltUp_nbui.mask(BuiltUp_nbui).clip(per_urb), {'palette': indiceBinPalette}, 'Cobertura artificial (manmade) cf. nbui')
right_layer = geemap.ee_tile_layer(NonBuiltUp_nbui.mask(NonBuiltUp_nbui).clip(per_urb), {'palette': indiceBinPaletteR}, 'Cobertura natural cf. nbui')

text = 'Área impermeável (manmade) cf. NBUI: ' + str( round(square_kilometers1,2) )+ ' km² (2000) <BR> Área permeável cf. NBUI: ' + str( round(square_kilometers0,2) )+ ' km² (2000) <BR> Área urbana cf. IPPUL: ' + str(area_urbana)+ ' km² (2019)'
params = {
    'fontsize': 18,
    'fontcolor': 'black',
    'bold': False,
    'padding': '10px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '5px',
    'position': 'bottomleft',
}
mapa11 = geemap.Map()
mapa11.centerObject(per_urb, 12)
mapa11.split_map(left_layer, right_layer)
mapa11.add_text(text, **params)
mapa11.addLayer(visual.clip(per_urb), {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3}, 'Imagem a classificar')
mapa11



In [ ]:
# Quantificação IBI
nPixelsIBI1 = imagemIndicesMedianos.select('ibi').gt(thresholdIBI).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=per_urb,
    scale=30,
    maxPixels=1e9
)

#print('Área com atributo 1',  nPixelsIBI1.getInfo())

nPixelsIBI1=nPixelsIBI1.getInfo()
areaIBI1 = ( nPixelsIBI1['ibi'])/1000


In [ ]:
# Classificação binária (Otsu threshold) pelo IBI num painel subdividido
BuiltUp_ibi = IBIMediano.select('ibi').gt(thresholdIBI)
NonBuiltUp_ibi = IBIMediano.select('ibi').lt(thresholdIBI)

left_layer = geemap.ee_tile_layer(BuiltUp_ibi.mask(BuiltUp_ibi).clip(per_urb), {'palette': indiceBinPalette}, 'Cobertura artificial (manmade) cf. ibi')
right_layer = geemap.ee_tile_layer(NonBuiltUp_ibi.mask(NonBuiltUp_ibi).clip(per_urb), {'palette': indiceBinPaletteR}, 'Cobertura natural cf. ibi')

mapa12 = geemap.Map()
mapa12.centerObject(per_urb, 12)
mapa12.split_map(left_layer, right_layer)
text = 'Área artificial (manmade) cf. IBI: ' + str( round(areaIBI1,0) )+ 'km² <BR> Área urbana cf. IPPUL: ' + str(area_urbana)+ ' km² '
params = {
    'fontsize': 18,
    'fontcolor': 'black',
    'bold': False,
    'padding': '10px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '5px',
    'position': 'bottomleft',
}

mapa12.add_text(text, **params)
mapa12

In [ ]:
# Quantificação NDBI
nPixelsNDBI1 = imagemIndicesMedianos.select('ndbi').gt(thresholdNDBI).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=per_urb,
    scale=30,
    maxPixels=1e9
)

#print('Área com atributo 1',  nPixelsNDBI1.getInfo())

nPixelsNDBI1=nPixelsNDBI1.getInfo()
areaNDBI1 = ( nPixelsNDBI1['ndbi'])/1000


nPixelsNDBI0 = imagemIndicesMedianos.select('ndbi').lt(thresholdNDBI).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=per_urb,
    scale=30,
    maxPixels=1e9
)

#print('Área com atributo 0',  areaNaoNDBIm2.getInfo())

nPixelsNDBI0=nPixelsNDBI0.getInfo()
areaNDBI0 = ( nPixelsNDBI0['ndbi'])/1000



In [ ]:
#  Classificação binária (Otsu threshold) pelo NDBI num painel subdividido
BuiltUp_ndbi = NDBIMediano.select('ndbi').gt(thresholdNDBI)
NonBuiltUp_ndbi = NDBIMediano.select('ndbi').lt(thresholdNDBI)

left_layer = geemap.ee_tile_layer(BuiltUp_ndbi.mask(BuiltUp_ndbi).clip(per_urb), {'palette': indiceBinPalette}, 'Cobertura artificial (manmade) cf. mdbi')

right_layer = geemap.ee_tile_layer(NonBuiltUp_ndbi.mask(NonBuiltUp_ndbi).clip(per_urb), {'palette': indiceBinPaletteR}, 'Cobertura natural cf. mdbi')

mapa13 = geemap.Map()
mapa13.centerObject(per_urb, 12)

mapa13.split_map(left_layer, right_layer)

text = 'Área artificial (manmade) cf. NDBI: ' + str( round(areaNDBI1,0) )+ ' km² <BR> Área urbana cf. IPPUL: ' + str(area_urbana)+ ' km² '
params = {
    'fontsize': 18,
    'fontcolor': 'black',
    'bold': False,
    'padding': '10px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '5px',
    'position': 'bottomleft',
}

mapa13.add_text(text, **params)
mapa13



In [ ]:
var sparcs = ee.ImageCollection('projects/gmap/datasets/manual_qaMasks/sparcs_masks');

var sparcIm = ee.Image(sparcs.toList(100).get(1));

var l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_RT_TOA").filterBounds(sparcIm.geometry());
var l8Image = l8.filterDate(sparcIm.date().advance(-6,'second'),sparcIm.date().advance(6,'second'));

var img = ee.Image(l8Image.first()).select(['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B9']);

Map.centerObject(img);
Map.addLayer(ee.Image(img).clip(sparcIm.geometry()),{min:0,max:0.3000,bands:"B4,B3,B2"},"landsat");

Map.addLayer(sparcs.select("b1"),{min:0,max:1},'sparc clouds');
Map.addLayer(sparcs.select("b2"),{min:0,max:1},'sparc shadow');
Map.addLayer(sparcs.select("b3"),{min:0,max:1},'sparc ice ');
Map.addLayer(sparcs.select("b4"),{min:0,max:1},'sparc water');